#### 3.3. Ending a game
A key concept in computer Go is self-play. In self-play, you usually start with a weak Go-playing agent, have it play against itself, and use the game results to build a stronger bot. In [chapter 4](chapter-4), you’ll use self-play to evaluate board positions. In chapters 9 through 12, you’ll use self-play to evaluate individual moves and the algorithms that selected them.

To take advantage of this technique, you need to make sure your self-play games end. Human games end when neither player can gain an advantage with their next move. This is a tricky concept even for humans. Beginners often end games by playing hopeless moves in the opponent’s territory, or watching their opponent cut into what they believed to be solid territory. For computers, it’s even more difficult. If our bot continues playing as long as legal moves remain available, it’ll eventually fill up its own liberties and lose all of its stones.

You could think up some heuristics to help the bot finish the game in a sensible manner. For example:

- Don’t play in a region that’s completely surrounded by stones of the same color.
- Don’t play a stone that would have only one liberty.
- Always capture an opposing stone with only one liberty.

Unfortunately, __all of those rules are too strict__. If our bots followed these rules, strong opponents would take advantage to kill groups that should live, rescue groups that should die, or simply gain a better position. Generally, our handcrafted rules should restrict the bot’s options as little as possible, so that more sophisticated algorithms are free to learn the advanced tactics.

To solve this problem, you can look to the history of the game. In ancient times, the winner was simply the player with the most stones on the board. Players would end the game by filling every point they could, leaving only eyes for their groups empty. This could make the end of a game drag out for a long time, so players came up with a way to speed it up. If black clearly controlled a region of the board (any white stone played there would eventually get captured), there was no need for black to fill that region with stones. The players would just agree to count that area for black. This is where the concept of territory came from, and over centuries the rules evolved so that territory was counted explicitly.

Scoring in this method avoids the question of what is territory and what isn’t, but you still have to prevent your bot from killing its own stones; see figure 3.8.

__Figure 3.8. White has two eyes in the corner, at A and B, and shouldn’t place a stone at either point. Otherwise, black can capture the whole group. Your naive bot won’t be allowed to fill its own eye.__
<p align="center">
     <img src="https://drek4537l1klr.cloudfront.net/pumperla/Figures/03fig08.jpg" alt="string">
</p>

You’ll hardcode a rule that prevents the bot from filling in its own eyes, under the strictest possible definition. For our purposes, an __eye__ is an empty point where all adjacent points and at least three out of four diagonally adjacent points are filled with friendly stones.

```{note}
Experienced Go players may notice that the preceding definition of __eye__ will miss a valid eye in some cases. We’ll accept those errors to keep the implementation simple.
```

You have to create a special case for eyes on the edge of the board; in that case, all the diagonally adjacent points must contain friendly stones.

You create a new submodule of dlgo called __agent__ (by creating a new folder named agent and an empty \_\_init__.py file within that folder) and place the following `is_point_an_eye` function into a file called helpers.py.

In [1]:
# Listing 3.15. Is the given point on the board an eye?
from dlgo.gotypes import Point


def is_point_an_eye(board, point, color):
    if board.get(point) is not None:
        return False
    for neighbor in point.neighbors():
        if board.is_on_grid(neighbor):
            neighbor_color = board.get(neighbor)
            if neighbor_color != color:
                return False

    friendly_corners = 0
    off_board_corners = 0
    corners = [
        Point(point.row - 1, point.col - 1),
        Point(point.row - 1, point.col + 1),
        Point(point.row + 1, point.col - 1),
        Point(point.row + 1, point.col + 1),
    ]
    for corner in corners:
        if board.is_on_grid(corner):
            corner_color = board.get(corner)
            if corner_color == color:
                friendly_corners += 1
        else:
            off_board_corners += 1
    if off_board_corners > 0:
        return off_board_corners + friendly_corners == 4
    return friendly_corners >= 3



You aren’t explicitly concerned with determining the result of a game yet in this chapter, but counting points at the end of a game is definitely an important topic. Different tournaments and Go federations enforce slightly different rule sets. Throughout the book, you’ll have your bots follow the AGA rules for __area counting__, also known as __Chinese counting__. Although __Japanese rules__ are more popular in casual play, the AGA rules are a little easier for computers, and the rule differences rarely affect the outcome of a game.